In [1]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 27.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=f719a8b1c71f13d99ff484a3f93673139b4010d11f002733f5a96c8f04f0117e
  Stored in directory: /Users/danielmlow/Library/Caches/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [12]:
# clone repository

!git clone https://github.com/datavorous/YARS.git


Cloning into 'YARS'...
remote: Enumerating objects: 260, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 260 (delta 69), reused 48 (delta 30), pack-reused 148 (from 1)
Receiving objects: 100% (260/260), 1.30 MiB | 15.35 MiB/s, done.
Resolving deltas: 100% (124/124), done.


In [9]:
!pip install uv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 47.1 MB/s eta 0:00:00a 0:00:01


In [13]:
from yars import YARS
from utils import display_results, download_image

miner = YARS()

ModuleNotFoundError: No module named 'yars'

In [10]:
subreddit_posts = miner.fetch_subreddit_posts("bullying", limit=11, category="top", time_filter="week")
display_results(subreddit_posts, "EarthPorn SUBREDDIT New Posts")

NameError: name 'miner' is not defined

In [2]:
# Import packages to use in extracting out data.
import os
import sys
import numpy as np
import pandas as pd
import json
from pprint import pprint
import requests
import time
import random


from langdetect import detect

from datetime import datetime, date, timedelta
import string
from scipy import stats

In [3]:
# Cell that contains helpful functions for datetime operations and parsing the json output from the url query
def date2timestamp(date):
    '''
    "01/12/2011"
    '''
    return int(time.mktime(datetime.strptime(date, "%Y/%m/%d").timetuple()))

def next_day(date):
  tmrw = datetime.strptime(date, "%Y/%m/%d") + timedelta(days=1)
  return tmrw.strftime('%Y/%m/%d')

def list_of_days(start_date, end_date):
    '''
    start_date = date(2019, 1, 19)   # start date
    end_date = date(2019, 3, 22)   # end date
    '''
    delta = end_date - start_date       # as timedelta
    days = []
    for i in range(delta.days + 1):
        day = start_date + timedelta(days=i)
        days.append(str(day).replace('-', '/'))
    return days


In [4]:
# Specify output directory based on location of script

# output dir if drive has been mounted:
output_dir = '/content/drive/My Drive/ML4HC_Final_Project/data/input/raw_post/'

In [5]:
# Method used to get posts
def scrape_reddit(output_dir, subreddit, timeframe, date_start, date_end, size = 1000):
    '''
    size = {1,1000} #amount of posts
    '''
    start = date2timestamp(date_start)
    end = date2timestamp(date_end)
    # use the pushshift api to extract out data
    url = 'https://api.pushshift.io/reddit/search/submission/?subreddit={}&sort=desc&sort_type=created_utc&after={}&before={}&size={}'.format(subreddit,start, end, size)
    print(url)
    try:
        posts = requests.get(url)
        posts = posts.json()
        posts = posts['data']
    except:
        time.sleep(30)
        posts = requests.get(url)
        posts = posts.json()
        posts = posts['data']

    df = pd.DataFrame(columns=['subreddit', 'author', 'date', 'post'])
    
    for post in posts:
      if 'selftext' in post: # check if selftext parameter exists
        text = post['selftext']
        if text != "" and  text != '[removed]' and '[deleted]' not in text: # further check if selftext is not empty
          try: 
            if detect(text) == 'en': # check if text is in english - if the language detected is not in the langdetect library, then continue to the next post
              df = df.append({'subreddit': subreddit, 'author': post['author'], 'date': date_start, 'post': post['title'] + ' ' + post['selftext']}, ignore_index=True)
          except:
            continue
    return df

In [7]:
subreddits = ['bullying']

In [ ]:
# # Subreddits to scrape:

# subreddits = ['suicidewatch',
#               'depression',
#               'ptsd',
#               'anxiety',
#               'socialanxiety',
#               'healthanxiety',
#               'bipolarreddit',
#               'bpd',
#               'schizophrenia',
#               'paranoia',
#               'EDAnonymous',
#               'alcoholism',
#               'addiction',
#               'adhd',
#               'mindfulness', 
#               'psychosis', 
#               'mentalillness',
#               'mentalhealth', #end of subreddits used in both pre- and mid-pandemic analysis
#               'meditation',
#               'personalfinance',
#               'jokes',
#               'teaching',
#               'relationships',
#               'legaladvice',
#               'fitness',
#               'parenting',
#               'COVID19_support',
#               'lonely',
#               'autism',
#               'divorce',
#               'guns',
#               'economy',
#               'ForeverAlone',
#               'politics',
#               'ukpolitics',
#               'CanadaPolitics',
#               'conspiracy',
#               'India',
#               'Canada',
#               'unitedkingdom',
#               'UKPersonalFinance',
#               'MakeNewFriendsHere'# end of subreddits only used for mid-pandemic analysis/clustering
#               ]

In [8]:
# Extract out reddit data given specifications of subreddits + pre pandemic distinction, as well as window of days to extract from

days = list_of_days(date(2023,11,1), date(2024,11,1))
size = 1000

timeframe = 'pre'

for subreddit in subreddits:
  subreddit_df = pd.DataFrame(columns=['subreddit', 'author', 'date', 'post'])
  days_local = list(days)
  while subreddit_df.shape[0] < 30000 and days_local:
    idx = random.randint(0, len(days_local)-1)
    date_start = days_local.pop(idx)
    date_end = next_day(date_start)
    df = scrape_reddit(output_dir, subreddit, timeframe, date_start, date_end, size = size)
    subreddit_df = pd.concat([subreddit_df, df])
    time.sleep(0.5)
    print(subreddit)
    print(subreddit_df.shape)
  # subreddit_df.to_csv(os.path.join(output_dir, '{}_{}.csv'.format(subreddit, timeframe)), index=False)

https://api.pushshift.io/reddit/search/submission/?subreddit=bullying&sort=desc&sort_type=created_utc&after=1703998800&before=1704085200&size=1000


KeyError: 'data'

In [ ]:
# Extract out reddit data given specifications of subreddits from 2018, as well as window of days to extract from

days = list_of_days(date(2018,1,1), date(2018,12,31))
size = 1000

timeframe = '2018'

for subreddit in subreddits[3:]:
  subreddit_df = pd.DataFrame(columns=['subreddit', 'author', 'date', 'post'])
  days_local = list(days)
  while subreddit_df.shape[0] < 30000 and days_local:
    idx = random.randint(0, len(days_local)-1)
    date_start = days_local.pop(idx)
    date_end = next_day(date_start)
    df = scrape_reddit(output_dir, subreddit, timeframe, date_start, date_end, size = size)
    subreddit_df = pd.concat([subreddit_df, df])
    time.sleep(0.5)
    print(subreddit)
    print(subreddit_df.shape)
  subreddit_df.to_csv(os.path.join(output_dir, '{}_{}.csv'.format(subreddit, timeframe)), index=False)

https://api.pushshift.io/reddit/search/submission/?subreddit=anxiety&sort=desc&sort_type=created_utc&after=1533254400&before=1533340800&size=1000
anxiety
(150, 4)
https://api.pushshift.io/reddit/search/submission/?subreddit=anxiety&sort=desc&sort_type=created_utc&after=1520985600&before=1521072000&size=1000
anxiety
(269, 4)
https://api.pushshift.io/reddit/search/submission/?subreddit=anxiety&sort=desc&sort_type=created_utc&after=1521676800&before=1521763200&size=1000
anxiety
(421, 4)
https://api.pushshift.io/reddit/search/submission/?subreddit=anxiety&sort=desc&sort_type=created_utc&after=1546128000&before=1546214400&size=1000
anxiety
(570, 4)
https://api.pushshift.io/reddit/search/submission/?subreddit=anxiety&sort=desc&sort_type=created_utc&after=1535587200&before=1535673600&size=1000
anxiety
(745, 4)
https://api.pushshift.io/reddit/search/submission/?subreddit=anxiety&sort=desc&sort_type=created_utc&after=1520121600&before=1520208000&size=1000
anxiety
(861, 4)
https://api.pushshift.

JSONDecodeError: ignored

In [ ]:
# Extract out reddit data given specifications of subreddits + post pandemic distinction, as well as window of days to extract from

days = list_of_days(date(2018,1,1), date(2018,4,20))
size = 1000

timeframe = '2018'

for subreddit in subreddits:
  subreddit_df = pd.DataFrame(columns=['subreddit', 'author', 'date', 'post'])
  days_local = list(days)
  for date_start in days_local:
    date_end = next_day(date_start)
    df = scrape_reddit(output_dir, subreddit, timeframe, date_start, date_end, size = size)
    subreddit_df = pd.concat([subreddit_df, df])
    time.sleep(0.5)
    print(subreddit)
    print(subreddit_df.shape)
  subreddit_df.to_csv(os.path.join(output_dir, '{}_{}.csv'.format(subreddit, timeframe)), index=False)

Streaming output truncated to the last 5000 lines.
(0, 4)
https://api.pushshift.io/reddit/search/submission/?subreddit=COVID19_support&sort=desc&sort_type=created_utc&after=1522886400&before=1522972800&size=1000
COVID19_support
(0, 4)
https://api.pushshift.io/reddit/search/submission/?subreddit=COVID19_support&sort=desc&sort_type=created_utc&after=1522972800&before=1523059200&size=1000
COVID19_support
(0, 4)
https://api.pushshift.io/reddit/search/submission/?subreddit=COVID19_support&sort=desc&sort_type=created_utc&after=1523059200&before=1523145600&size=1000
COVID19_support
(0, 4)
https://api.pushshift.io/reddit/search/submission/?subreddit=COVID19_support&sort=desc&sort_type=created_utc&after=1523145600&before=1523232000&size=1000
COVID19_support
(0, 4)
https://api.pushshift.io/reddit/search/submission/?subreddit=COVID19_support&sort=desc&sort_type=created_utc&after=1523232000&before=1523318400&size=1000
COVID19_support
(0, 4)
https://api.pushshift.io/reddit/search/submission/?subred